In [1]:
import time
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn as nn
from torch.autograd import Variable
import torch

#加上transforms
transform=transforms.Compose([
    transforms.Resize((512,512), interpolation=3), #缩放图片，保持长宽比不变，最短边的长为224像素,
#     transforms.CenterCrop(512), #从中间切出 224*224的图片    
#     transforms.RandomResizedCrop(512),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #将图片转换为Tensor,归一化至[0,1]
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])

train_dataset = ImageFolder('D:\\WorkSpace\\jupyter\\DataSet\\MangoLeaf\\train',transform=transform)num_classes = 2
test_dataset = ImageFolder('D:\\WorkSpace\\jupyter\\DataSet\\MangoLeaf\\test',transform=transform)
num_classes = 2

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#输出第0张图片的大小
print(train_dataset[0][0].size)

<built-in method size of Tensor object at 0x0000024098A59948>


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# print(torch.cuda.get_device_name(device))

cuda:0


In [4]:
from torch.utils.data import DataLoader

torch.manual_seed(1)
BATCH_SIZE = 2

train_loader = DataLoader(
    dataset=train_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True,# 设置随机洗牌
    num_workers=5# 加载数据的进程个数
)

test_loader = DataLoader(
    dataset=test_dataset,# TensorDataset类型数据集
    batch_size=1,# mini batch size
    shuffle=True,# 设置随机洗牌
    num_workers=5# 加载数据的进程个数
)

In [5]:
batch = next(iter(train_loader))

In [6]:
print(batch[0].size())
print(batch[1].size())

torch.Size([2, 3, 512, 512])
torch.Size([2])


In [7]:
# 两层卷积
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 使用序列工具快速构建
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(128 * 128 * 32, 2)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)  # reshape
        out = self.fc(out)
        return out

In [8]:
# 3x3 卷积
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(128 * 128, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out




In [9]:
from torchvision import models
# cnn = CNN()
cnn = ResNet(ResidualBlock, [2, 2, 2, 2])
print(cnn)
if torch.cuda.is_available():
    cnn = cnn.cuda()

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [10]:
num_epochs = 10
LEARNING_RATE = 0.01

# 选择损失函数和优化方法
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=LEARNING_RATE)

# 将数据处理成Variable, 如果有GPU, 可以转成cuda形式
def get_variable(x):
    x = Variable(x)
    return x.cuda() if torch.cuda.is_available() else x

In [11]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    eval_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images = get_variable(images)
            labels = get_variable(labels)

            outputs = model(images)
            eval_loss += loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    eval_loss = eval_loss / total
#     print('Accuracy of the network on the test images: %d %%' % (
#         test_acc))
    return test_acc, eval_loss

In [12]:
losses = []
accuracies = []
test_accuracies = []
# set the model to train mode initially
cnn.train()


for epoch in range(num_epochs):
    since = time.time()
    running_loss = 0.0
    running_correct = 0.0
    for step, (images, labels) in enumerate(train_loader):
        images = get_variable(images)
        labels = get_variable(labels)

        cnn.train()
        outputs = cnn(images)
        loss = loss_func(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # calculate the loss/acc later
        running_loss += loss.item()
        running_correct += (labels==predicted).sum().item()

    epoch_duration = time.time()-since
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100/BATCH_SIZE*running_correct/len(train_loader)
    print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
    
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)

    # switch the model to eval mode to evaluate on test data
    cnn.eval()
    test_acc, test_loss = eval_model(cnn)
    print("eval_loss: %.4f, eval_acc: %.4f" % (test_loss, test_acc))
    test_accuracies.append(test_acc)
    
     # re-set the model to train mode after validating
    cnn.train()
#     scheduler.step(test_acc)
    since = time.time()
    
print('Finished Training')


Epoch 1, duration: 116 s, loss: 5.1246, acc: 72.0874
eval_loss: 0.0338, eval_acc: 100.0000
Epoch 2, duration: 120 s, loss: 0.4679, acc: 81.7961
eval_loss: 0.4703, eval_acc: 78.2609
Epoch 3, duration: 124 s, loss: 0.5556, acc: 79.1262
eval_loss: 0.4350, eval_acc: 78.2609
Epoch 4, duration: 126 s, loss: 0.4330, acc: 81.5534
eval_loss: 0.3707, eval_acc: 82.6087
Epoch 5, duration: 126 s, loss: 0.4098, acc: 83.0097
eval_loss: 0.4132, eval_acc: 86.9565
Epoch 6, duration: 128 s, loss: 0.4163, acc: 81.3107
eval_loss: 0.4421, eval_acc: 82.6087
Epoch 7, duration: 129 s, loss: 0.3968, acc: 84.2233
eval_loss: 0.3073, eval_acc: 82.6087
Epoch 8, duration: 131 s, loss: 0.3649, acc: 87.3786
eval_loss: 0.2791, eval_acc: 82.6087
Epoch 9, duration: 130 s, loss: 0.3753, acc: 83.9806
eval_loss: 0.2894, eval_acc: 86.9565
Epoch 10, duration: 137 s, loss: 0.2968, acc: 89.3204
eval_loss: 0.3317, eval_acc: 86.9565
Finished Training
